In [3]:

import cv2, PIL, os, sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from cv2 import aruco
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as mpl
import pandas as pd
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")

%matplotlib nbagg
%load_ext autoreload
%autoreload 2

In [4]:
def create_char_board(ARUCO_SIZE:int, SQUARES_VERTICALLY:int, SQUARES_HORIZONTALLY:int, SQUARE_LENGTH:float, MARKER_LENGTH:float, SKIP:bool=False, LEGACY:bool = False):
    if ARUCO_SIZE == 5:
        ARUCO_DICT = cv2.aruco.DICT_5X5_250
    elif ARUCO_SIZE == 6:
        ARUCO_DICT = cv2.aruco.DICT_6X6_250
    else:
        raise ValueError("aruco dict size must be either 5 or 6 (otherwise change code manually)")
    LENGTH_PX = 500   # total length of the page in pixels
    MARGIN_PX = 20    # size of the margin in pixels
    dictionary = cv2.aruco.getPredefinedDictionary(ARUCO_DICT)
    if SKIP:
        dictionary.bytesList = dictionary.bytesList[100:]
    board = cv2.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)
    board.setLegacyPattern(LEGACY)
    params = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, params)
    size_ratio = SQUARES_HORIZONTALLY / SQUARES_VERTICALLY
    img = cv2.aruco.CharucoBoard.generateImage(board, (LENGTH_PX, int(LENGTH_PX*size_ratio)), marginSize=MARGIN_PX)
    cv2.imshow("img", img)
    cv2.waitKey()
    # cv2.imwrite(SAVE_NAME, img)
    cv2.destroyAllWindows()
    return board, dictionary, params, detector

# PARAMS:
a4_small = dict(ARUCO_SIZE = 5,
                SQUARES_VERTICALLY = 11,
                SQUARES_HORIZONTALLY = 15,
                SQUARE_LENGTH = 0.015,
                MARKER_LENGTH = 0.011,
                SKIP = True,
                LEGACY = False
)
a4_big = dict(ARUCO_SIZE = 5,
              SQUARES_VERTICALLY = 6,
              SQUARES_HORIZONTALLY = 9,
              SQUARE_LENGTH = 0.015,
              MARKER_LENGTH = 0.011,
              SKIP = False,
              LEGACY = False
)
a3 = dict(ARUCO_SIZE = 6,
          SQUARES_VERTICALLY = 7,
          SQUARES_HORIZONTALLY = 10,
          SQUARE_LENGTH = 0.0394,
          MARKER_LENGTH = 0.0394 * 0.75,
          SKIP = False,
          LEGACY = True
)

char_board, char_dict, char_params, char_detector = create_char_board(**a3)

In [5]:
charuco_folder = "Test Data/charuco_temp_folder"

In [13]:
def load_intrinsic(video_folder:str, every_x_frames:int):
    char_frames = []
    for i, file in enumerate(os.listdir(video_folder)):
        current_frame = -1
        charuco_vid_file_1 = f"{video_folder}/{file}"
        video = cv2.VideoCapture(charuco_vid_file_1)
        grabbed, frame = video.read()
        while grabbed:
            current_frame += 1
            if not current_frame % every_x_frames:
                char_frames.append(frame)
            grabbed, frame = video.read()
    print("number of pics loaded: ", len(char_frames))
    return char_frames

char_folder = f"{charuco_folder}/intrinsic_vid"
every_xth_frame = 20
charuco_frames = load_intrinsic(video_folder = char_folder, every_x_frames = every_xth_frame)

number of pics loaded:  244


In [14]:
def save_intrinsic(folder, frames):   # must manually empty out first to prevent residuals
    if not len(os.listdir(folder)) == len(frames):
        for i, img in enumerate(frames):
            cv2.imwrite(f"{folder}/pic_{i+1}.png", img)
            
save_intrinsic(f"{charuco_folder}/intrinsic_pic", charuco_frames)

In [10]:
def save_extrinsic(char_folder:str):
    vid_folder = f"{char_folder}/extrinsic_vid"
    for i, file in enumerate(os.listdir(vid_folder)):
        vid_file = f"{vid_folder}/{file}"
        video = cv2.VideoCapture(vid_file)
        grabbed, frame = video.read()
        print(frame.shape)
        cv2.imwrite(f"{char_folder}/extrinsic_pic/pic_{i+1}.png", frame)

In [11]:
save_extrinsic(charuco_folder)

(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)


In [ ]:
# ADDING OTHER PICS TO EXTRINSIC FOLDER MANUALLY
# cv2.imshow("test", charuco_frames[0])
# cv2.waitKey()
# cv2.destroyAllWindows()
# cv2.imwrite(f"{charuco_folder}/extrinsic_pic/pic_{1}.png", charuco_frames[0])

In [17]:
all_charuco_corners = []
all_charuco_ids = []
image_shape = "placeholder"
used_images_index = []
used_images_holder = []
for i, image in enumerate(charuco_frames):
    if image_shape == "placeholder":
        image_shape = image.shape[:2]
    image_copy = image.copy()
    marker_corners, marker_ids, rejectedCandidates = char_detector.detectMarkers(image_copy)
    # If at least one marker is detected
    if marker_ids is not None:
        cv2.aruco.drawDetectedMarkers(image_copy, marker_corners, marker_ids)
        charuco_retval, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, image, char_board)
        if charuco_retval and len(charuco_ids) > 10 and len(charuco_corners > 10):
            all_charuco_corners.append(charuco_corners)
            all_charuco_ids.append(charuco_ids)
            used_images_index.append(i)
            used_images_holder.append(image_copy)
retval, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.aruco.calibrateCameraCharuco(all_charuco_corners.copy(), all_charuco_ids.copy(), char_board, image_shape, None, None)
# print([len(x) for x in all_charuco_corners])
# print(used_images_index)
# print(len(used_images_index))
# Calibrate camera

[16, 29, 17, 19, 25, 20, 30, 24, 15, 15, 12, 14, 13, 17, 13, 16, 32, 45, 49, 51, 46, 42, 35, 29, 18, 15, 36, 40, 18, 15, 48, 38]
[2, 3, 4, 17, 18, 19, 20, 21, 22, 23, 24, 85, 86, 87, 88, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 237, 238, 239, 240, 241, 242, 243]


In [1]:
retval, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.aruco.calibrateCameraCharuco(all_charuco_corners.copy(), all_charuco_ids.copy(), char_board, image_shape, None, None)

NameError: name 'cv2' is not defined

In [20]:
print(f"retval:  {retval}")
# print(f"retval_real: {retval_real}")

retval:  15.818112165181818


In [22]:
dist_coeffs = dist_coeffs[0]


In [23]:
image_used = cv2.imread(f"{charuco_folder}/chessboard_pic/0.png")
# cv2.imshow("show", image_used)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [24]:
image_points = np.float32(([1007, 39], [1897, 1017], [33, 45], [1878, 90], [194, 73], [981, 15], [909, 1021]))
world_points = np.float32(((-13, -22.6, 0), (-11.2, 7.7, 0), (31, -95.9, 0), (107.5, 89, 0), (31.3, -89, 0), (79.6, -21.4, 0), (-14.5, -11.5, 0)))

In [25]:
retval, rvec, tvec = cv2.solvePnP(
    objectPoints=world_points,
    imagePoints=image_points,
    cameraMatrix=camera_matrix,
    distCoeffs=dist_coeffs
)

In [26]:
if retval:
    print("rvec:", rvec)
    print("tvec:", tvec)
    print("retval: ", retval)
else:
    print("solvePnP failed to find a soldution")

rvec: [[-2.5906028 ]
 [-0.32594126]
 [-1.3323899 ]]
tvec: [[-111.40295905]
 [-263.27664583]
 [ 381.28338623]]
retval:  True


In [160]:
print(tvec[0])

[[ 0.29711573]
 [-0.31109331]
 [ 1.02956   ]]


In [27]:
rotation_matrix, _ = cv2.Rodrigues(rvec)
extrinsic_matrix = np.concatenate((rotation_matrix, tvec), axis=1)
intrinsic_matrix = camera_matrix
camera_coordinates = -np.matrix(rotation_matrix).T * np.matrix(tvec)
print(camera_coordinates)
# camera: ongeveer 155.5, 57, 76 

[[-222.44748945]
 [-186.60590091]
 [ 377.88567626]]


In [23]:
# cv2.imshow("image", actual_used_images[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [84]:
# undistorted_image_1 = actual_used_images[0].copy()
# cv2.aruco.drawDetectedMarkers(undistorted_image_1, all_charuco_corners[0], all_charuco_ids[0])
image_used = charuco_frames[0].copy()
undistorted_image_1 = cv2.undistort(image_used.copy(), camera_matrix, dist_coeffs)
# cv2.drawFrameAxes(undistorted_image_1, camera_matrix, dist_coeffs, rvec_real, tvec_real, length=0.1, thickness=15)
cv2.imshow("aha", undistorted_image_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
projected_points, _ = cv2.projectPoints(world_points, rvec, tvec, camera_matrix, dist_coeffs)
projected_points = projected_points.squeeze()
copy = image_used.copy()
# copy = cv2.undistort(copy, camera_matrix, dist_coeffs)
for (i, pt) in enumerate(projected_points):
    cv2.circle(copy, (int(pt[0]), int(pt[1])), 5, (0, 0, 255), -1)
    cv2.circle(copy, (int(image_points[i, 0]), int(image_points[i, 1])), 5, (255, 0, 0), -1)
    # cv2.putText(img=copy, text=f"{i}", org=(int(pt[0]), int(pt[1])),fontFace=1,fontScale=1.5, color=(0, 0, 255), thickness=1)
    # cv2.putText(img=copy, text=f"{i}", org=(int(image_points[i, 0]), int(image_points[i, 1])), fontFace=1, fontScale=1.5, color=(255, 0, 0), thickness=1)

cv2.imwrite(f"{charuco_folder}/Shit.png", copy)
cv2.imshow('Projected Points', copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'copy'

In [127]:
reprojection_error = np.linalg.norm(image_points - projected_points, axis=1)
mean_error = np.mean(reprojection_error)
std_dev_error = np.sqrt(np.sum((reprojection_error - mean_error) ** 2) / len(reprojection_error))
# print(reprojection_error)
print(mean_error)
print(std_dev_error)

18.664637
11.570609743000176


In [30]:
# # assuming we have correct intrinsic and extrinsic matrices, We now want to project to top view.
# output_image_size = 4 * image_used.shape[:-2][::-1]
# world_points_for_top = np.array([(4, 0, 57.7), (4, 142, 57.7), (130, 30, 57.7), (140, 80, 57.7)], dtype=np.float32)
# image_points_for_top, _ = cv2.projectPoints(world_points_for_top, rvec_real, tvec_real, camera_matrix, dist_coeffs)
# copy = image_used.copy()
# for (i, pt) in enumerate(image_points_for_top):
#     pt = pt[0]
#     cv2.circle(copy, (int(pt[0]), int(pt[1])), 5, (0, 0, 255), -1)
#     # cv2.circle(copy, (int([i, 0]), int(image_points_for_top[i, 1])), 5, (255, 0, 0), -1)
# print(image_points_for_top[2:])
# 
# cv2.imshow('Projected Points', copy)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [31]:
print(camera_matrix)

[[7.87352602e+03 0.00000000e+00 1.41368070e+03]
 [0.00000000e+00 3.81181316e+03 1.03606940e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [36]:
img = charuco_frames[0]
# 
# vid_file = "../../Test Data/Triangulation_test/Charuco-07-06-2024/video"
# cap = cv2.VideoCapture(f"{vid_file}/{os.listdir(vid_file)[1]}")
# grabbed, img = cap.read()
cv2.imshow("img", img)
# Get the current working directory
script_dir = %pwd

# Save the original image
original_img_path = os.path.join(script_dir, 'input.png')
cv2.imwrite(original_img_path, img)

# Perform camera calibration and obtain new camera matrix
h, w = img.shape[:2]
newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (w, h), 1, (w, h))

# Undistort the image
dst_undistorted = cv2.undistort(img, camera_matrix, dist_coeffs, None, newCameraMatrix)

# Get the new dimensions after undistortion and cropping
x, y, w, h = roi
new_w = w - x
new_h = h - y

# Crop the undistorted image
dst_cropped = dst_undistorted[y:y+h, x:x+w]

# Save the undistorted and cropped image
undistorted_cropped_img_path = os.path.join(script_dir, 'undistorted_and_cropped.png')
cv2.imwrite(undistorted_cropped_img_path, dst_cropped)

# Perform undistortion with remapping
mapx, mapy = cv2.initUndistortRectifyMap(camera_matrix, dist_coeffs, None, newCameraMatrix, (w, h), cv2.CV_32FC1)
dst_remap = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

# Crop the remapped image
dst_remap_cropped = dst_remap[y:y+h, x:x+w]

# Save the remapped and cropped image
remapped_cropped_img_path = os.path.join(script_dir, 'remapped_and_cropped.png')
cv2.imshow(remapped_cropped_img_path, dst_remap_cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [49]:
# Open video
# cap = cv2.VideoCapture("videos/smoke/needswarping.mp4")
# worldspace = np.float32([(15.8, 10.8, 50), (29.2, 3.9, 50), (36.4, 47.8, 50), (5.2, 59.6, 50)])
# imagespace = np.float32([[923, 1487], [142, 908], [1047, 564], [13, 393]])
worldspace = world_points
imagespace = image_points
vid_file = "Test Data/charuco_temp_folder/chessboard_vid"
cap = cv2.VideoCapture(f"{vid_file}/{os.listdir(vid_file)[0]}")

# Get frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Extract x, y coordinates and assume the plane normal points towards Z-axis
world_xy = worldspace[:, :2]
plane_normal = np.array([0, 0, 1])

# Calculate the constant term (d) in the plane equation (Ax + By + Cz + d = 0)
d = -worldspace[0][2]  # Use any point on the plane

# Create the plane equation representation
plane_equation = np.concatenate((plane_normal, [d]))

# Define a new virtual camera position
camera_distance = 100  # Set a reasonable distance
camera_position = np.array([80, 80, -camera_distance])

# Function to project a point onto the plane
def project_onto_plane(point, plane_equation):
    return point - (np.dot(point - camera_position, plane_equation[:3]) / np.linalg.norm(plane_equation[:3])**2) * plane_equation[:3]

# Project world points onto the plane
projected_worldspace = np.array([project_onto_plane(point, plane_equation) for point in worldspace])

# Find minimum and maximum coordinates of projected points
min_x, min_y = projected_worldspace[:, 0].min(), projected_worldspace[:, 1].min()
max_x, max_y = projected_worldspace[:, 0].max(), projected_worldspace[:, 1].max()

# Scale and translate to fit the frame size
scaling_factor = min(frame_width / (max_x - min_x), frame_height / (max_y - min_y))
translation = np.array([-min_x * scaling_factor, -min_y * scaling_factor])

# Function to transform the points
def transform_point(point):
    return (point[:2] * scaling_factor) + translation

# Transform the points
transformed_points = np.array([transform_point(point) for point in projected_worldspace])

# Calculate homography matrix from the original image points to the transformed points
homography_matrix, _ = cv2.findHomography(imagespace, transformed_points)

# Main loop to process the video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply the homography transformation to the frame
    warped_frame = cv2.warpPerspective(frame, homography_matrix, (frame_width, frame_height))

    # Display the original and warped frames
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Warped Frame', cv2.resize(warped_frame, (warped_frame.shape[1] // 2, warped_frame.shape[0] // 2)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
# image_points = np.array([
#     [170, 409],
#     [107, 49],
#     [580, 64],
#     [631, 65],
#     [1029, 78],
#     [1078, 80],
#     [120, 177],
#     [587, 186],
#     [612, 186],
#     [1015, 190],
#     [150, 392],
#     [582, 364],
#     [606, 363],
#     [1172, 367],
#     [380, 396],
#     [637, 390],
#     [717, 396],
#     [791, 386],
#     [944, 383],
#     [372, 409],
#     [583, 418],
#     [608, 417],
#     [633, 402],
#     [794, 398],
#     [950, 394],
#     [1024, 394],
#     [88, 516],
#     [599, 493],
#     [17, 632],
#     [170, 622],
#     [333, 613],
#     [489, 602],
#     [621, 594]
# ], dtype=np.float32)
# 
# world_points = np.array([
#     [0, 0, 0],
#     [0, 0, 57.7],
#     [0, 67, 57.7],
#     [0, 75, 57.7],
#     [0, 142, 57.7],
#     [0, 150, 57.7],
#     [0, -2, 39],
#     [0, 69, 39],
#     [0, 73, 39],
#     [0, 144, 39],
#     [0, -2, 7],
#     [0, 69, 7],
#     [0, 73, 7],
#     [110.2, 89.7, 57.7],
#     [0, 33.5, 2],
#     [0, 78.5, 2],
#     [112.5, 62.75, 57.7],
#     [0, 108.5, 2],
#     [0, 138.5, 2],
#     [2, 33.5, 0],
#     [7, 69, 0],
#     [7, 73, 0],
#     [2, 78, 0],
#     [2, 108.5, 0],
#     [2, 139, 0],
#     [2, 153, 0],
#     [31.1, -2, 0],
#     [31.1, 69, 0],
#     [55.2, -2, 0],
#     [55.2, 14.75, 0],
#     [55.2, 33.5, 0],
#     [55.2, 52.25, 0],
#     [55.2, 69, 0]
# ], dtype=np.float32)


In [ ]:
# retval_real, _, _, rvec_real, tvec_real = cv2.aruco.calibrateCameraCharuco([all_charuco_corners.copy()[-1]], [all_charuco_ids.copy()[-1]], char_board, actual_used_images[-1].shape[:2], camera_matrix, dist_coeffs)
# rvec_real, tvec_real = rvec_real[0], tvec_real[0]
# all_charuco_corners = []
# all_charuco_ids = []
# image_shape = "placeholder"
# used_images = []
# test_img = charuco_frames[76]
# cv2.imshow("test_img", test_img)
# cv2.waitKey()
# cv2.destroyAllWindows()
# for i, image in enumerate(test_img):
#     if image_shape == "placeholder":
#         image_shape = image.shape[:2]
#     image_copy = image.copy()
#     marker_corners, marker_ids, rejectedCandidates = char_detector.detectMarkers(image_copy)
#     # If at least one marker is detected
#     if marker_ids is not None:
#         cv2.aruco.drawDetectedMarkers(image_copy, marker_corners, marker_ids)
#         charuco_retval, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, image, char_board)
#         if charuco_retval and len(charuco_ids) > 8 and len(charuco_corners > 8):
#             all_charuco_corners.append(charuco_corners)
#             all_charuco_ids.append(charuco_ids)
#             used_images.append(i)
# print([len(x) for x in all_charuco_corners])
# retval, rvec, tvec = cv2.aruco.calibrateCameraCharuco(all_charuco_corners.copy(), all_charuco_ids.copy(), char_board, image_shape, camera_matrix, dist_coeffs)
# counter = 0
# for i, image in enumerate(charuco_frames):
#     copy_image = image.copy()
#     undistorted_image = cv2.undistort(image.copy(), camera_matrix, dist_coeffs)
#     if i in used_images:
#         cv2.aruco.drawDetectedCornersCharuco(copy_image, all_charuco_corners[counter], all_charuco_ids[counter], cornerColor=(255, 0, 0))
#         counter += 1
#     cv2.imshow("original image", copy_image)
#     cv2.imshow('Undistorted Image',  undistorted_image)
#     cv2.waitKey(0)
# cv2.destroyAllWindows()
# charuco_images_folder = "../../Test Data/Triangulation_test/Charuco-30-05-2024/Charuco_images"
# # for i, image in enumerate(charuco_frames):
# #     cv2.imwrite(f"{charuco_images_folder}/{i+3}.png", cv2.undistort(image.copy(), camera_matrix, dist_coeffs))
# cv2.imwrite(f"{charuco_images_folder}/test_20.png", charuco_frames[17].copy())
# real_world_points = np.array([(147, -71, 0), (147, 71, 0), (147, 71, -57.8), (147, -71, -57.8), (147, 0, 0), (147, 0, -57.8), (147, 35.5, -28.9)], dtype=np.float32)
# # pixel_points = np.array([[212, 44][::-1], [1175, 71][::-1], [1086, 428][::-1], [266, 386][::-1]], dtype=np.float32)
# pixel_points = np.array([[208, 43][::-1], [1182, 69][::-1], [1095, 429][::-1], [261, 387][::-1], [677, 57][::-1], [666, 410][::-1], [900, 253][::-1]], dtype=np.float32)
# image_points = np.array([[165, 636], [679, 641], [677,30], [62, 66], [1001, 241], [68,1038], [1032, 980]], dtype=np.float32)
# world_points = np.array([[0, 0, 0], [0, 0.8, 0], [0, 0.8, 0.8], [0, 0, 0.8], [2, 4.5, 0], [0.9, 0, 0], [0.9, 1.5, 0]], dtype=np.float32)
# retval, rvec, tvec = cv2.calibrateCamera(objectPoints=[real_world_points], imagePoints=[pixel_points], imageSize=image_used.shape[:2][::-1], cameraMatrix=np.float32(camera_matrix), distCoeffs=np.float32(dist_coeffs))

In [ ]:
# def get_custom_axes(cam, dist, rvec, tvec, length=1.):
#     # set array to configure custom axes
#     axes = np.array([[0., 0., 0.], [length, 0., 0.], [0., length, 0.], [0., 0., -length]])
#     axes_to_img, _ = cv2.projectPoints(axes, rvec, tvec, cam, dist)
#     return np.floor(np.squeeze(axes_to_img)).astype(int)
# 
# def draw_axes(img, axes, thickness=5):
#     img = cv2.line(img, axes[0], axes[0+1], (0,0,255), thickness)
#     img = cv2.line(img, axes[0], axes[1+1], (0,255,0), thickness)
#     img = cv2.line(img, axes[0], axes[2+1], (255,0,0), thickness)
#     return img
# 
# def testttt(image):
#     image_copy = image.copy()
#     marker_corners, marker_ids, rejectedCandidates = char_detector.detectMarkers(image_copy)
#     cv2.aruco.drawDetectedMarkers(image_copy, marker_corners, marker_ids)
#     charuco_retval, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, image, char_board)
#     cv2.aruco.drawDetectedCornersCharuco(image_copy, charuco_corners, charuco_ids)
#     valid, rvec, tvec = aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, char_board, camera_matrix, dist_coeffs, np.empty(1), np.empty(1), useExtrinsicGuess=False)
#     
#     # image_copy = cv2.drawFrameAxes(image_copy, camera_matrix, dist_coeffs, rvec, tvec, 0.08) 
#     image_copy = draw_axes(image_copy, get_custom_axes(camera_matrix, dist_coeffs, rvec, tvec, 0.02))
#     image_copy = cv2.resize(image_copy, (image_copy.shape[1] // 2, image_copy.shape[0] // 2), cv2.INTER_AREA)
#     cv2.imshow("f", image_copy)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
# testttt(charuco_frames[0])